In [1]:
import pandas as pd
import numpy as np
import json
import ast
import collections
import re
import pprint
import csv
from tqdm import tqdm
import pymongo
import operator
from ast import literal_eval

In [2]:
df_metadata= pd.read_csv("Resources/movies_metadata.csv", low_memory=False)
df_metadata.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies_cols=["id", "title", "budget", "genres", "production_countries", "revenue"]
df_movies=df_metadata[movies_cols].copy()
df_movies.head()

,id,title,budget,genres,production_countries,revenue
0,862,Toy Story,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0
1,8844,Jumanji,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",262797249.0
2,15602,Grumpier Old Men,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
3,31357,Waiting to Exhale,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",81452156.0
4,11862,Father of the Bride Part II,0,"[{'id': 35, 'name': 'Comedy'}]","[{'iso_3166_1': 'US', 'name': 'United States o...",76578911.0


In [4]:
renamed={"production_countries": "productionCountries"}
df_movies = df_movies.rename(columns=renamed)
df_movies

,id,title,budget,genres,productionCountries,revenue
0,862,Toy Story,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0
1,8844,Jumanji,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'iso_3166_1': 'US', 'name': 'United States o...",262797249.0
2,15602,Grumpier Old Men,0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
3,31357,Waiting to Exhale,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",81452156.0
4,11862,Father of the Bride Part II,0,"[{'id': 35, 'name': 'Comedy'}]","[{'iso_3166_1': 'US', 'name': 'United States o...",76578911.0
...,...,...,...,...,...,...
45461,439050,Subdue,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...","[{'iso_3166_1': 'IR', 'name': 'Iran'}]",0.0
45462,111109,Century of Birthing,0,"[{'id': 18, 'name': 'Drama'}]","[{'iso_3166_1': 'PH', 'name': 'Philippines'}]",0.0
45463,67758,Betrayal,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...","[{'iso_3166_1': 'US', 'name': 'United States o...",0.0
45464,227506,Satan Triumphant,0,[],"[{'iso_3166_1': 'RU', 'name': 'Russia'}]",0.0


In [5]:
print("Start Shape: " + str(df_movies.shape))
df_movies.dropna(inplace=True)
df_movies.drop_duplicates(inplace=True)
print("End Shape: " + str(df_movies.shape))

Start Shape: (45466, 6)
End Shape: (45430, 6)


In [6]:
df_movies["budget"] = df_movies["budget"].astype(float)

In [7]:
converted_values = df_movies["productionCountries"].apply(literal_eval)
converted_values.head()

0    [{'iso_3166_1': 'US', 'name': 'United States o...
1    [{'iso_3166_1': 'US', 'name': 'United States o...
2    [{'iso_3166_1': 'US', 'name': 'United States o...
3    [{'iso_3166_1': 'US', 'name': 'United States o...
4    [{'iso_3166_1': 'US', 'name': 'United States o...
Name: productionCountries, dtype: object

In [8]:
extracted_countries = []

for item in converted_values.values:
    if len(item) == 0:
        # print('Item empty, skipping')
        extracted_countries.append('')
    else:
        # print(item[0]['name'])
        extracted_countries.append(item[0]['name'])

In [9]:
df_movies["productionCountries"] = extracted_countries
df_movies.head()

,id,title,budget,genres,productionCountries,revenue
0,862,Toy Story,30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",United States of America,373554033.0
1,8844,Jumanji,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",United States of America,262797249.0
2,15602,Grumpier Old Men,0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",United States of America,0.0
3,31357,Waiting to Exhale,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",United States of America,81452156.0
4,11862,Father of the Bride Part II,0.0,"[{'id': 35, 'name': 'Comedy'}]",United States of America,76578911.0


In [10]:
pd.set_option('display.max_columns', None)
df_movies["id"] = pd.to_numeric(df_movies["id"].replace([np.inf, -np.inf], np.nan).dropna(), downcast="integer", errors="coerce")
df_movies = df_movies.dropna()
df_movies["id"] = df_movies["id"].astype(int)

In [11]:
print("End Shape: " + str(df_movies.shape))

End Shape: (45430, 6)


In [12]:
# Subroutine that does the following things:
# Grabs the JSON text from the column key spcified and returns an Object
# Assumes there is a list of objects and iterates over them
# Grab the data for the key and append it to a local list
# Once we are done iterating over the object, replace the element that
# we started with with a colon seperated list on the key words
def getJSONDictElements(df, colKey, dictKey, seperator, totalDict):
    with tqdm(total=df.shape[0]) as pbar:
        for index, row in df.iterrows():
            pbar.update(1)
            json_string = row[colKey]
            if not isinstance(json_string, str):
                # skip lines that aren't strings
                continue
            if "{" not in json_string:
                # skip lines that aren't dict's
                continue
            obj = ast.literal_eval(json_string)
            elementList = []
            for item in obj:
            # pprint.pprint(item)
                if len(item[dictKey]) > 0:
                    elementList.append(item[dictKey])
                    if item[dictKey] not in totalDict:
                        totalDict[item[dictKey]] = 1
                    else:
                        totalDict[item[dictKey]] += 1
            df.at[index, colKey] = seperator.join(elementList)

In [13]:
file = "Resources/genre_list.csv"
total_genres = {}
getJSONDictElements(df_movies, "genres", "name", ":", total_genres)
with open(file, 'w', newline='') as csvFile:
    csvwriter = csv.writer(csvFile)
    for key in sorted(total_genres.keys()):
        if total_genres[key] < 5:
            # skip the production company names
            continue
        csvwriter.writerow([key, total_genres[key]])

df_movies.head()

100%|██████████| 45430/45430 [00:04<00:00, 10028.70it/s]


,id,title,budget,genres,productionCountries,revenue
0,862,Toy Story,30000000.0,Animation:Comedy:Family,United States of America,373554033.0
1,8844,Jumanji,65000000.0,Adventure:Fantasy:Family,United States of America,262797249.0
2,15602,Grumpier Old Men,0.0,Romance:Comedy,United States of America,0.0
3,31357,Waiting to Exhale,16000000.0,Comedy:Drama:Romance,United States of America,81452156.0
4,11862,Father of the Bride Part II,0.0,Comedy,United States of America,76578911.0


In [14]:
file = "Resources/keywords.csv"
df_key = pd.read_csv(file)
df_key.head()

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."
4,11862,"[{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n..."


In [15]:
total_key_words = {}
getJSONDictElements(df_key, "keywords", "name", ":", total_key_words)
df_key.head()

100%|██████████| 46419/46419 [00:04<00:00, 9339.65it/s] 


,id,keywords
0,862,jealousy:toy:boy:friendship:friends:rivalry:bo...
1,8844,board game:disappearance:based on children's b...
2,15602,fishing:best friend:duringcreditsstinger:old men
3,31357,based on novel:interracial relationship:single...
4,11862,baby:midlife crisis:confidence:aging:daughter:...


In [16]:
joined_df = pd.merge(left=df_movies, right=df_key, left_on="id", right_on="id")
print(joined_df.head())
print("End Shape: " + str(joined_df.shape))

      id                        title      budget                    genres  \
0    862                    Toy Story  30000000.0   Animation:Comedy:Family   
1   8844                      Jumanji  65000000.0  Adventure:Fantasy:Family   
2  15602             Grumpier Old Men         0.0            Romance:Comedy   
3  31357            Waiting to Exhale  16000000.0      Comedy:Drama:Romance   
4  11862  Father of the Bride Part II         0.0                    Comedy   

        productionCountries      revenue  \
0  United States of America  373554033.0   
1  United States of America  262797249.0   
2  United States of America          0.0   
3  United States of America   81452156.0   
4  United States of America   76578911.0   

                                            keywords  
0  jealousy:toy:boy:friendship:friends:rivalry:bo...  
1  board game:disappearance:based on children's b...  
2   fishing:best friend:duringcreditsstinger:old men  
3  based on novel:interracial relationsh

In [17]:
joined_df = joined_df.dropna(how='any')
print("End Shape: " + str(joined_df.shape))

End Shape: (46415, 7)


In [18]:
joined_df.drop_duplicates(inplace=True)
print("End Shape: " + str(joined_df.shape))

End Shape: (45429, 7)


In [19]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
# Declare the database
db = client.movies_db
collection = db['movie_table']
collection.drop() # drop the table to start fresh
collection = db['movie_table']

# Convert the joined_df to a dict
data_dict = joined_df.to_dict("records")

collection.insert_many(data_dict)

In [20]:
collection.count()

<ipython-input-20-7c4bc5758cb0>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  collection.count()


45429